In [17]:
import numpy as np
import matplotlib.pyplot as plt
import itertools #used to generate subsets
from scipy.ndimage import gaussian_filter
from skimage.segmentation import watershed

# from sknetwork.path import distance
# import sknetwork as skn
# from scipy import sparse
# import networkx as nx can be used if we need more advanced graphs data structure

In [ ]:
#get image and define the image space
g = #image_here
image_range = zip(range(image.shape[0]),range(image.shape[1]))

In [2]:
def B_delta(p,delta): #rename suggestion: delta ball
    B = set()
    for q in image_range:
        if np.linalg.norm(p-q)<= delta:
            B.add(q)
    return B

### Algorithm I

In [19]:
def generate_subgraphs(adjacency):
    #TODO Fehlerabfangen: adjacency symmetric etc.
    dim = adjacency.shape[0]
    vertices = range(dim)
    S = {}
    for length in range(2, dim+1):
        for subs in list(itertools.combinations(vertices, length)):
            sub_adj = np.zeros((dim, dim))
            if all(adjacency[np.ix_(subs, subs)][k].any(0) > 0 for k in range(len(subs))):
                S[subs] = adjacency[np.ix_(subs, subs)]
               #ise this if we need adjacency matrix with original dim 
               # for nodes in subs:
               #     for neighbours in subs:
               #         sub_adj[nodes][neighbours] = adjacency[nodes][neighbours]
               #         sub_adj[neighbours][nodes] = adjacency[nodes][neighbours]
               # S[subs] = sub_adj
    return S

In [ ]:
#TODO
def is_connected(adjacency):
    #will do when we're sure what simply connected means

In [96]:
# Work in Progress to generate all connected subgraphs via a DFS approach.
# Current Problem: error that U.pop() wants to pop an empty list, but idk why it's empty

def generate_connected_subgraphs(adjacency, node): #adjacency needs to be a numpy array
    #TODO: Fehlerabfangen, dass adjacency gültige Form hat
    dim = adjacency.shape[0]
    if node > dim:
        ValueError("Nodes are counted starting 0.")
    T = np.ones(dim)    
    S = np.zeros(dim)
    U = []
    U.append(node)
    T[node] = 0
    
    adj_graph = np.zeros((dim, dim))
    Matrices = []
    
    while U != [] or T != np.zeros(dim):
        if U != []:
            v = U[-1]
            for i in np.where(adjacency[v] == 1)[0]:
                if T[i] == 1:
                    U.append(i)
                    T[i] = 0
                    adj_graph[v][i] = 1
                    adj_graph[i][v] = 1
                    Matrices.append(adj_graph)
                else:
                    S[i] = 1
                    U.pop()
    return Matrices

In [14]:
def generate_fragments(img,std_dev,int_treshold,min_seed_dist,max_search_depth,max_frag_dist):
    g = gaussian_filter(img,std_dev)
    g_copy = np.copy(g)
    
    #Fehlerabfangen
    
    if std_dev < 0:
        raise ValueError("smoothing strength needs to be positive")
    if int_threshold > 1 or int_threshold < 0:
        raise ValueError("relative intensity threshold needs to be between 0 and 1")
    if min_seed_dist < 1:
        raise ValueError("min seed distance needs to be >= 1")
    if max_search_depth < 0:
        raise ValueError("maximum search error needs to be positive")
    if max_frag_dist < 0:
        raise ValueError("maximum fragments distance needs zo be positive")
        
    
    PI = []
    for p in image_range:
        B = B_delta(p,min_seed_dist)
        if (g[p]== max(g[q] for q in B)) and (1-int_treshold)*g[p]>=min(g[q] for q in B): 
            PI.append(p) 
            g_copy[p] = len(PI) 
        else:
            g_copy[p]=0   
    
    Omega = []
    Labels = watershed(img,markers=g_copy)
    for label in range(1,len(PI)+1):
        w = np.argwhere(Labels==label) #Das sind nur die indizies der Region
        Omega.append(w)
        
    Epsilon = np.zeros((len(Omega), len(Omega)))
    for i,_ in enumerate(Omega[:-1]):
        for j,_ in enumerate(Omega[i+1:]):
            #adjacency
            #local intensity peaks r(w)??
            if np.linalg.norm(PI[i] - PI[j]) <= max_frag_dist:
                Epsilon[i][j] = 1
                Epsilon[j][i] = 1
             
    
   # Epsilon = set()
   # for i,w in enumerate(Omega[:-1]):
   #     for j,v in enumerate(Omega[i+1:],start=i+1):
   #         #What is adjacency?
   #         if (x+(e,0) in v or x+(0,e) in v for x in w for e in (1,-1)) and np.linalg.norm(PI[i] - PI[j]) <= max_frag_dist:
   #             Epsilon.add({w,v})
            
    S = set()
    subgraphs_set = generate_subgraphs(Epsilon)
    #TODO: add simply connected if-Abfrage
    for i in range(Epsilon.shape[0]): #iterate over all graph vertices
    for subs in subgraphs_set.keys(): 
        if i in range(len(subs)): #find all subgraphs that have i as a node
            dis_matrix = distance(subgraphs_set[subs], method = 'D') #return distance matrix of graph subs
            if dis_matrix[i].max(0) <= h:
                for k in subs:
                    S.add(k)
                #del subgraphs_set[subs] #this changes size during itertaion, ungünstig
                #find a way to not iterate over subgraphs we've already worked with
    return Omega, S

###  testing out graphs

In [45]:
adjacency = np.array([[0, 1, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [0, 1, 0, 0]])
#adjacency = sparse.csr_matrix(adjacency)
S = set()
subgraphs_set = generate_subgraphs(adjacency)
for i in range(adjacency.shape[0]): #iterate over all graph vertices
    for subs in subgraphs_set.keys(): 
        if i in range(len(subs)): #find all subgraphs that have i as a node
            dis_matrix = distance(subgraphs_set[subs], method = 'D') #return distance matrix of graph subs
            if dis_matrix[i].max(0) <= 3:
                for k in subs:
                    S.add(k)
                #del subgraphs_set[subs] #this changes size during itertaion, ungünstig
                #find a way to not iterate over subgraphs we've already worked with
S

{0, 1, 2, 3}

In [110]:
for x in generate_subgraphs(adjacency).keys():
    if 1 in x:
        print(x)

(0, 1)
(1, 2)
(1, 3)
(0, 1, 2)
(0, 1, 3)
(1, 2, 3)
(0, 1, 2, 3)


### Algorithm II

In [ ]:
def global_solution(f,alpha,Omega,S):
    n = len(S) #stimmt n?
    u = np.zeros(n)
    V = Omega
    Z = S
    while V != ():
#work in progress
        c = np.empty(n)
        for k in range(len(S)):
            c[k] = (f[k] + alpha) / #abs value?
        k_min = argmin(c)
        u[k_min],V = 1, #v\{Z}?
    
    while Z!= set():
        k_prim = 